In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file
csv_path = "crops2.csv"
df = pd.read_csv(csv_path)

# Function to calculate suitability score for a crop
def calculate_suitability(row, temp, hum, soil, ph):
    score = 0.0
    # Weight for each parameter (equal weighting)
    weight = 0.25

    # Temperature score
    temp_min, temp_max = map(float, row['Optimal_Temperature_Range (°C)'].split('-'))
    if temp_min <= temp <= temp_max:
        score += weight * 1.0
    elif abs(temp - temp_min) <= 2 or abs(temp - temp_max) <= 2:
        score += weight * 0.5  # Partial score if within 2°C

    # Humidity score
    hum_min, hum_max = map(float, row['Optimal_Humidity_Range (%)'].split('-'))
    if hum_min <= hum <= hum_max:
        score += weight * 1.0
    elif abs(hum - hum_min) <= 5 or abs(hum - hum_max) <= 5:
        score += weight * 0.5  # Partial score if within 5%

    # Soil moisture score
    soil_min, soil_max = map(float, row['Optimal_Soil_Moisture_Range (%)'].split('-'))
    if soil_min <= soil <= soil_max:
        score += weight * 1.0
    elif abs(soil - soil_min) <= 5 or abs(soil - soil_max) <= 5:
        score += weight * 0.5  # Partial score if within 5%

    # pH score
    ph_min, ph_max = map(float, row['Optimal_pH_Range'].split('-'))
    if ph_min <= ph <= ph_max:
        score += weight * 1.0
    elif abs(ph - ph_min) <= 0.2 or abs(ph - ph_max) <= 0.2:
        score += weight * 0.5  # Partial score if within 0.2 pH

    return score

# Function to find the best crop(s)
def find_best_crop(df, temp, hum, soil, ph, region=None):
    # Filter by region if specified
    if region:
        df = df[df['Region'].str.contains(region, case=False, na=False)]

    # Calculate suitability scores
    df['Suitability_Score'] = df.apply(
        lambda row: calculate_suitability(row, temp, hum, soil, ph), axis=1
    )

    # Sort by suitability score
    df_sorted = df.sort_values(by='Suitability_Score', ascending=False)

    # Return top crop(s)
    top_crops = df_sorted[['Crop', 'Region', 'Suitability_Score',
                           'Optimal_Temperature_Range (°C)', 'Optimal_Humidity_Range (%)',
                           'Optimal_Soil_Moisture_Range (%)', 'Optimal_pH_Range']]

    return top_crops

# User input: Sensor readings (default: Eastern UP conditions)
print("Enter sensor readings (or press Enter to use default Eastern UP values):")
try:
    temp = float(input("Temperature (°C, default 25.2): ") or 25.2)
    hum = float(input("Humidity (%, default 55.5): ") or 55.5)
    soil = float(input("Soil Moisture (%, default 47.2): ") or 47.2)
    ph = float(input("pH (default 7.4): ") or 7.4)
except ValueError:
    print("Invalid input! Using default values.")
    temp, hum, soil, ph = 25.2, 55.5, 47.2, 7.4

# Optional: Filter by region (e.g., 'North' for Eastern UP)
region = input("Enter region (e.g., North, South, or leave blank for all India): ").strip() or None

# Find best crops
top_crops = find_best_crop(df, temp, hum, soil, ph, region)

# Print results
print("\nTop Crops for Conditions (Temperature: {:.1f}°C, Humidity: {:.1f}%, Soil Moisture: {:.1f}%, pH: {:.1f})".format(temp, hum, soil, ph))
if region:
    print(f"Filtered for Region: {region}")
print(top_crops.head(10))  # Show top 10 crops

# Visualization: Bar plot of suitability scores
plt.figure(figsize=(12, 6))
sns.barplot(data=top_crops.head(15), x='Suitability_Score', y='Crop', hue='Region')
plt.title('Top 15 Crops by Suitability Score')
plt.xlabel('Suitability Score (0-1)')
plt.ylabel('Crop')
plt.tight_layout()
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')